In [ ]:
# Setup - Add src to path
import sys
from pathlib import Path

# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Setup complete!")
print(f"Project root: {project_root}")

In [ ]:
# Import from src modules
from config import *
from data_loader import load_all_data, merge_datasets
from bucket_calculator import compute_avg_j, create_auxiliary_file, compute_normalized_volume
from eda_analysis import (
    analyze_data_quality, analyze_bucket_distribution, 
    analyze_erosion_curves, run_full_eda
)

print("✅ Modules imported successfully!")

## 1. Load Data

In [ ]:
# Load training data
volume, generics, medicine = load_all_data(train=True)
merged = merge_datasets(volume, generics, medicine)

print(f"\n📊 Dataset Summary:")
print(f"   Total rows: {len(merged):,}")
print(f"   Unique brands: {merged[['country', 'brand_name']].drop_duplicates().shape[0]}")
print(f"   Columns: {merged.columns.tolist()}")

In [ ]:
# Create auxiliary file with buckets
aux_df = create_auxiliary_file(merged, save=False)
avg_j = aux_df[['country', 'brand_name', 'avg_vol']].copy()

# Add normalized volume
merged_norm = compute_normalized_volume(merged, avg_j)

print(f"\n📊 Auxiliary file created")
print(f"   Bucket distribution:")
print(aux_df['bucket'].value_counts())

## 2. Data Quality Analysis

In [ ]:
# Run data quality analysis
quality_results = analyze_data_quality(merged)

# Display missing values
if quality_results['missing_values'].sum() > 0:
    print("⚠️ Missing values found:")
    print(quality_results['missing_values'][quality_results['missing_values'] > 0])
else:
    print("✅ No missing values!")

In [ ]:
# Visualize volume distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Raw volume distribution
axes[0].hist(merged['volume'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Volume')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Volume Distribution (Raw)')
axes[0].set_yscale('log')

# Log volume distribution
log_volume = np.log1p(merged['volume'])
axes[1].hist(log_volume, bins=50, alpha=0.7, color='coral', edgecolor='black')
axes[1].set_xlabel('Log(Volume + 1)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Volume Distribution (Log Scale)')

# Normalized volume distribution
valid_norm = merged_norm['vol_norm'].dropna()
valid_norm = valid_norm[valid_norm < 5]  # Clip outliers for visualization
axes[2].hist(valid_norm, bins=50, alpha=0.7, color='seagreen', edgecolor='black')
axes[2].set_xlabel('Normalized Volume (vol/avg_j)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Normalized Volume Distribution')
axes[2].axvline(x=1, color='red', linestyle='--', label='Pre-entry average')
axes[2].axvline(x=0.25, color='orange', linestyle='--', label='Bucket 1 threshold')
axes[2].legend()

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'volume_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"✅ Saved to {FIGURES_DIR / 'volume_distributions.png'}")

## 3. Bucket Distribution Analysis

In [ ]:
# Bucket distribution analysis
bucket_results = analyze_bucket_distribution(aux_df)

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart of bucket distribution
bucket_counts = aux_df['bucket'].value_counts().sort_index()
colors = ['#ff6b6b', '#4ecdc4']
axes[0].pie(bucket_counts, labels=[f'Bucket {i}\n({c} brands)' for i, c in bucket_counts.items()],
           autopct='%1.1f%%', colors=colors, explode=[0.05, 0],
           shadow=True, startangle=90)
axes[0].set_title('Bucket Distribution\n(Bucket 1 = High Erosion, 2× Weight)', fontsize=12)

# Mean erosion by bucket
erosion_by_bucket = aux_df.groupby('bucket')['mean_erosion'].agg(['mean', 'std', 'count'])
bars = axes[1].bar(erosion_by_bucket.index, erosion_by_bucket['mean'], 
                   yerr=erosion_by_bucket['std'], capsize=5, color=colors, edgecolor='black')
axes[1].axhline(y=0.25, color='red', linestyle='--', label='Bucket 1 threshold (0.25)')
axes[1].set_xlabel('Bucket')
axes[1].set_ylabel('Mean Erosion (Normalized Volume)')
axes[1].set_title('Mean Erosion by Bucket', fontsize=12)
axes[1].legend()
axes[1].set_xticks([1, 2])

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'bucket_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"✅ Saved to {FIGURES_DIR / 'bucket_distribution.png'}")

In [ ]:
# Mean erosion histogram
fig, ax = plt.subplots(figsize=(10, 5))

# Plot histogram
ax.hist(aux_df['mean_erosion'].dropna(), bins=50, alpha=0.7, color='steelblue', edgecolor='black')
ax.axvline(x=0.25, color='red', linestyle='--', linewidth=2, label='Bucket 1 threshold (≤0.25)')
ax.axvline(x=aux_df['mean_erosion'].median(), color='green', linestyle=':', linewidth=2, 
           label=f'Median: {aux_df["mean_erosion"].median():.3f}')

ax.set_xlabel('Mean Erosion (24-month average normalized volume)', fontsize=11)
ax.set_ylabel('Number of Brands', fontsize=11)
ax.set_title('Distribution of Mean Erosion Across All Brands', fontsize=12)
ax.legend(fontsize=10)

# Add annotation
bucket1_count = (aux_df['mean_erosion'] <= 0.25).sum()
ax.annotate(f'Bucket 1: {bucket1_count} brands\n(High erosion, 2× weight)', 
            xy=(0.15, ax.get_ylim()[1]*0.8), fontsize=10, 
            bbox=dict(boxstyle='round', facecolor='#ff6b6b', alpha=0.3))

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'mean_erosion_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Generic Erosion Curves

In [ ]:
# Analyze erosion curves
erosion_results = analyze_erosion_curves(merged_norm, aux_df)

print("📊 Erosion Curve Statistics:")
print(erosion_results['summary'])

In [ ]:
# Plot erosion curves by bucket
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Get post-entry data
post_entry = merged_norm[(merged_norm['months_postgx'] >= 0) & (merged_norm['months_postgx'] <= 23)].copy()
post_entry = post_entry.merge(aux_df[['country', 'brand_name', 'bucket']], on=['country', 'brand_name'])

# Average erosion curve by month and bucket
erosion_by_month = post_entry.groupby(['bucket', 'months_postgx'])['vol_norm'].mean().reset_index()

for bucket in [1, 2]:
    bucket_data = erosion_by_month[erosion_by_month['bucket'] == bucket]
    color = '#ff6b6b' if bucket == 1 else '#4ecdc4'
    label = f'Bucket {bucket} (High Erosion)' if bucket == 1 else f'Bucket {bucket} (Lower Erosion)'
    axes[0].plot(bucket_data['months_postgx'], bucket_data['vol_norm'], 
                 marker='o', linewidth=2, color=color, label=label)

axes[0].axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='Pre-entry average')
axes[0].axhline(y=0.25, color='red', linestyle=':', alpha=0.5, label='Bucket 1 threshold')
axes[0].set_xlabel('Months Post Generic Entry', fontsize=11)
axes[0].set_ylabel('Normalized Volume (vol/avg_j)', fontsize=11)
axes[0].set_title('Average Erosion Curves by Bucket', fontsize=12)
axes[0].legend(fontsize=9)
axes[0].set_xlim(-1, 24)
axes[0].set_ylim(0, 1.2)

# Sample individual curves
sample_brands = aux_df.sample(min(20, len(aux_df)))[['country', 'brand_name', 'bucket']]
sample_data = post_entry.merge(sample_brands, on=['country', 'brand_name', 'bucket'])

for (country, brand), group in sample_data.groupby(['country', 'brand_name']):
    bucket = group['bucket'].iloc[0]
    color = '#ff6b6b' if bucket == 1 else '#4ecdc4'
    alpha = 0.7 if bucket == 1 else 0.3
    axes[1].plot(group['months_postgx'], group['vol_norm'], alpha=alpha, color=color, linewidth=0.8)

axes[1].axhline(y=1, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Months Post Generic Entry', fontsize=11)
axes[1].set_ylabel('Normalized Volume', fontsize=11)
axes[1].set_title('Sample Individual Erosion Curves (Red=Bucket1, Teal=Bucket2)', fontsize=12)
axes[1].set_xlim(-1, 24)
axes[1].set_ylim(0, 2)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'erosion_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"✅ Saved to {FIGURES_DIR / 'erosion_curves.png'}")

## 5. Category Analysis

In [ ]:
# Analyze by therapeutic area
ther_analysis = merged.groupby('ther_area').agg({
    'volume': ['mean', 'std', 'count'],
    'brand_name': 'nunique'
}).round(2)
ther_analysis.columns = ['mean_volume', 'std_volume', 'n_records', 'n_brands']
ther_analysis = ther_analysis.sort_values('n_brands', ascending=False)

print("📊 Analysis by Therapeutic Area:")
print(ther_analysis.head(10))

In [ ]:
# Visualize therapeutic area distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Top 10 therapeutic areas by brand count
top_ther = ther_analysis.head(10)
axes[0].barh(range(len(top_ther)), top_ther['n_brands'], color='steelblue', edgecolor='black')
axes[0].set_yticks(range(len(top_ther)))
axes[0].set_yticklabels(top_ther.index)
axes[0].set_xlabel('Number of Brands')
axes[0].set_title('Top 10 Therapeutic Areas by Brand Count')
axes[0].invert_yaxis()

# Country distribution
country_counts = merged[['country', 'brand_name']].drop_duplicates()['country'].value_counts()
axes[1].bar(range(len(country_counts)), country_counts.values, color='coral', edgecolor='black')
axes[1].set_xticks(range(len(country_counts)))
axes[1].set_xticklabels(country_counts.index, rotation=45, ha='right')
axes[1].set_xlabel('Country')
axes[1].set_ylabel('Number of Brands')
axes[1].set_title('Brand Count by Country')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'category_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Competition Analysis

In [ ]:
# Analyze generic competition
competition = merged[merged['months_postgx'] >= 0].copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Number of generics over time
generics_over_time = competition.groupby('months_postgx')['num_generics'].mean()
axes[0].plot(generics_over_time.index, generics_over_time.values, marker='o', color='purple', linewidth=2)
axes[0].set_xlabel('Months Post Generic Entry')
axes[0].set_ylabel('Average Number of Generic Competitors')
axes[0].set_title('Generic Competition Growth Over Time')
axes[0].set_xlim(-1, 24)

# Correlation: num_generics vs normalized volume
sample = competition.merge(merged_norm[['country', 'brand_name', 'months_postgx', 'vol_norm']], 
                           on=['country', 'brand_name', 'months_postgx'])
sample = sample.dropna(subset=['vol_norm', 'num_generics'])
sample = sample[sample['vol_norm'] < 3]  # Remove outliers

axes[1].scatter(sample['num_generics'], sample['vol_norm'], alpha=0.1, s=10, color='steelblue')
axes[1].set_xlabel('Number of Generic Competitors')
axes[1].set_ylabel('Normalized Volume')
axes[1].set_title('Impact of Generic Competition on Brand Volume')

# Add trend line
z = np.polyfit(sample['num_generics'], sample['vol_norm'], 1)
p = np.poly1d(z)
x_line = np.linspace(sample['num_generics'].min(), sample['num_generics'].max(), 100)
axes[1].plot(x_line, p(x_line), 'r--', linewidth=2, label=f'Trend (slope={z[0]:.4f})')
axes[1].legend()

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'competition_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Summary Statistics

In [ ]:
# Final summary
print("="*60)
print("📊 EDA SUMMARY")
print("="*60)

print(f"\n📈 Dataset Statistics:")
print(f"   Total records: {len(merged):,}")
print(f"   Unique country-brand pairs: {merged[['country', 'brand_name']].drop_duplicates().shape[0]:,}")
print(f"   Countries: {merged['country'].nunique()}")
print(f"   Therapeutic areas: {merged['ther_area'].nunique()}")

print(f"\n🎯 Bucket Distribution:")
bucket_dist = aux_df['bucket'].value_counts().sort_index()
print(f"   Bucket 1 (High Erosion, 2× weight): {bucket_dist.get(1, 0)} brands ({bucket_dist.get(1, 0)/len(aux_df)*100:.1f}%)")
print(f"   Bucket 2 (Lower Erosion): {bucket_dist.get(2, 0)} brands ({bucket_dist.get(2, 0)/len(aux_df)*100:.1f}%)")

print(f"\n📉 Erosion Statistics:")
print(f"   Mean erosion (all brands): {aux_df['mean_erosion'].mean():.3f}")
print(f"   Median erosion: {aux_df['mean_erosion'].median():.3f}")
print(f"   Bucket 1 mean erosion: {aux_df[aux_df['bucket']==1]['mean_erosion'].mean():.3f}")
print(f"   Bucket 2 mean erosion: {aux_df[aux_df['bucket']==2]['mean_erosion'].mean():.3f}")

print(f"\n✅ All figures saved to: {FIGURES_DIR}")
print("="*60)